參考網址: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets/

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 下載Porto Seguro's Safe Driver Prediction competition資料集
!gdown --id '1IY3IRmVx6v_sWB0zL8rbr0evDSo022oR' --output train.csv
!gdown --id '1tTM4d3pxwkS2U0qbHXpHP5QUoLIpYFvY' --output test.csv
!gdown --id '1y_14tK7Yx1Zo56ITLdSCutVlM4EeEaY_' --output sample_submission.csv

## Imbalanced datasets 不平衡的資料集

In [ ]:
df_train = pd.read_csv("train.csv")

target_count = df_train.target.value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)');

## The metric trap 測量陷阱

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# 去除'id'和'target'欄位
labels = df_train.columns[2:]

x = df_train[labels]
y = df_train['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("準確度: %.2f%%" % (accuracy * 100.0))

In [ ]:
model = XGBClassifier()
model.fit(x_train[['ps_calc_01']], y_train)
y_pred = model.predict(x_test[['ps_calc_01']])

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

## Confusion matrix 混淆矩陣

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

conf_mat = confusion_matrix(y_true = y_test, y_pred = y_pred)
print("Confusion matrix:\n", conf_mat)

In [ ]:
labels = ['Class 0', 'Class 1']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

## Resampling 重新抽樣

In [ ]:
# Class count
count_class_0, count_class_1 = df_train.target.value_counts()

# Divide by class
df_class_0 = df_train[df_train['target'] == 0]
df_class_1 = df_train[df_train['target'] == 1]

print(count_class_0,"0")
print(count_class_1,"1")

## Random under-sampling 隨機下抽樣

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis = 0)

print("Random under-sampling:")
print(df_test_under.target.value_counts())

df_test_under.target.value_counts().plot(kind = 'bar', title = "Count(target")

## Random over-sampling 隨機過抽樣

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace = True)
df_test_over = pd.concat([df_class_0, df_class_1_over])

print("Random over-Sampling:")
print(df_test_over.target.value_counts())

df_test_under.target.value_counts().plot(kind = 'bar', title = "Count(target")

## Python imbalanced-learn module imblearn套件

In [ ]:
import imblearn

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_classes=2, class_sep=1.5, weights=[0.9, 0.1],
    n_informative=3, n_redundant=1, flip_y=0,
    n_features=20, n_clusters_per_class=1,
    n_samples=100, random_state=10
)

df = pd.DataFrame(X)
df['target'] = y
df.target.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
def plot_2d_space(X, y, label='Classes'):   
    colors = ['#1F77B4', '#FF7F0E']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            X[y==l, 1],
            c=c, label=l, marker=m
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X = pca.fit_transform(X)

plot_2d_space(X, y, 'Imbalanced dataset (2 PCA components)')

## Random under-sampling and over-sampling with imbalanced-learn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(return_indices=True)
X_rus, y_rus, id_rus = rus.fit_sample(X, y)

print('Removed indexes:', id_rus)

plot_2d_space(X_rus, y_rus, 'Random under-sampling')

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X, y)

print(X_ros.shape[0] - X.shape[0], 'new random picked points')

plot_2d_space(X_ros, y_ros, 'Random over-sampling')

## Under-sampling: Cluster Centroids

In [ ]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(ratio={0: 10})
X_cc, y_cc = cc.fit_sample(X, y)

plot_2d_space(X_cc, y_cc, 'Cluster Centroids under-sampling')

## Over-sampling: SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X, y)

plot_2d_space(X_sm, y_sm, 'SMOTE over-sampling')

## Over-sampling followed by under-sampling

In [ ]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X, y)

plot_2d_space(X_smt, y_smt, 'SMOTE + Tomek links')